In [ ]:
from keras.preprocessing.image import ImageDataGenerator
import keras
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG19
import shutil
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score
import cv2
import matplotlib.pyplot as plt


In [ ]:
current_directory = os.getcwd()
train_dir = os.path.join(current_directory, "/home/abhi/MP/sign_data/merged_dataset/Train")
test_dir = os.path.join(current_directory, "/home/abhi/MP/sign_data/merged_dataset/Test")

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)


In [ ]:
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(200, 200),
                                                    batch_size=64,
                                                    class_mode='binary')


In [ ]:
test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=(200, 200),
                                                  batch_size=32,
                                                  class_mode='binary')

In [ ]:
import tensorflow as tf

class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.cnn1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(200, 200, 1))
        self.cnn2 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')
        self.cnn3 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu')

        self.flatten = tf.keras.layers.Flatten()

        self.dense1 = tf.keras.layers.Dense(512, activation='relu')
        self.dense2 = tf.keras.layers.Dense(1, activation='sigmoid')
        
    def call(self, inputs):
        x = self.cnn1(inputs)
        x = self.cnn2(x)
        x = self.cnn3(x)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        return x

In [ ]:
custom_cnn_model = MyModel()
custom_cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
custom_cnn_history = custom_cnn_model.fit(train_generator,
                                          steps_per_epoch=train_generator.samples // train_generator.batch_size,
                                          epochs=50,
                                          validation_data=test_generator,
                                          validation_steps=test_generator.samples // test_generator.batch_size,
                                          verbose=2)

In [ ]:
custom_cnn_predictions = custom_cnn_model.predict(test_generator)
